In [ ]:
import subprocess
import numpy as np

# Converts IP address into an integer
# Example: 192.168.1.10 = 192*256**3 + 168*256**2 + 1*256 + 10 = 3232235786
# Makes calculations easier and avoids string operations
def ip_to_int(ip):
    A, B, C, D = map(int, ip.split('.'))
    return A*256**3 + B*256**2 + C*256 + D

# Inverse of ip_to_int; converts integer back to IP address
# Example: 3232235786 → 192.168.1.10
def int_to_ip(num):
    A = num // 256**3
    num %= 256**3
    B = num // 256**2
    num %= 256**2
    C = num // 256
    D = num % 256
    return f"{A}.{B}.{C}.{D}"

# Converts CIDR notation into a subnet mask
# 2**32 = bits in whole IP address, so CIDR represents the number of network bits
def cidr_to_mask(cidr):
    mask_int = (2**32 - 1) - (2**(32 - cidr) - 1)
    return int_to_ip(mask_int), mask_int

# Optional: finds default class-based mask for a given IP integer
def ip_to_mask(ip_int):
    A = ip_int // 256**3
    if 1 <= A <= 126:
        return "255.0.0.0"
    elif 128 <= A <= 191:
        return "255.255.0.0"
    elif 192 <= A <= 223:
        return "255.255.255.0"
    else:
        return "Invalid address"

# Detects IP Address on device
def auto_detect_ip():
    print("Detecting host IP using Linux tools...")
    try:
        result = subprocess.check_output("ip -4 addr show", shell=True).decode()
        for line in result.splitlines():
            line = line.strip()
            # Match *any* inet IPv4 except loopback
            if line.startswith("inet ") and "127.0.0.1" not in line:
                parts = line.split()
                ip_cidr = parts[1]
                # If CIDR missing → assume /24 (most common)
                if "/" not in ip_cidr:
                    ip_cidr += "/24"
                print(f"Detected: {ip_cidr}")
                return ip_cidr
        print("No external IPv4 found.")
        return None
    except Exception as e:
        print(f"Error detecting IP: {e}")
        return None

#  Ping host
def ping_host(ip):
    try:
        result = subprocess.run(
            ["ping", "-c", "1", "-W", "1", ip],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        return result.returncode == 0
    except Exception:
        return False


#  Subnet Calculator
def calculate_subnet(ip_cidr, required_hosts):
    # Separates IP and CIDR
    try:
        ip_str, cidr_str = ip_cidr.split('/')
    except:
        raise ValueError(f"Invalid IP/CIDR format: {ip_cidr}")
    cidr = int(cidr_str)
    ip_int = ip_to_int(ip_str)

    # Checks if IP address is valid; avoids loopback/multicast/experimental addresses
    first_octet = int(ip_str.split('.')[0])
    if first_octet == 127 or first_octet > 223:
        print(f"IP address {ip_str} is reserved (loopback or multicast/experimental) and not typically used.")
        return None

    # Original mask
    orig_mask_str, orig_mask_int = cidr_to_mask(cidr)

    # Figuring out the host bits; loop until host bits >= required hosts
    host_bits = 0
    while (2**host_bits - 2) < required_hosts:
        host_bits += 1

    # Creates new CIDR based on host bits
    new_cidr = 32 - host_bits
    new_mask_str, new_mask_int = cidr_to_mask(new_cidr)

    # Creating the first network
    block_size = 2**host_bits
    network_int = ip_int - (ip_int % block_size)

    # Calculating number of subnets
    num_subnets = 2**max(0, (new_cidr - cidr))
    num_subnets = int(num_subnets)

    # Generates each subnet and stores in table
    rows = []
    for i in range(num_subnets):
        net = network_int + i * block_size
        brd = net + block_size - 1
        rows.append([
            str(i+1),  # Subnets start from 1
            int_to_ip(net),
            f"{int_to_ip(net+1)} - {int_to_ip(brd-1)}",
            int_to_ip(brd)
        ])

    # Display table using numpy for better viewing
    arr = np.array(rows)
    headers = ["Subnet", "Network ID", "Valid Range", "Broadcast ID"]
    col_widths = [max(len(headers[i]), max(len(str(x)) for x in arr[:, i])) for i in range(4)]

    header_line = "  ".join(headers[i].ljust(col_widths[i]) for i in range(4))
    print("\n" + header_line)
    print("-" * len(header_line))
    for row in arr:
        print("  ".join(row[i].ljust(col_widths[i]) for i in range(4)))

    # Summary of subnetting
    print("\n--- Summary ---")
    print(f"Given IP: {ip_str}")
    print(f"CIDR: {cidr}")
    print(f"Original Mask: {orig_mask_str}")
    print(f"New CIDR: {new_cidr}")
    print(f"New Mask: {new_mask_str}")
    print(f"Max Hosts/Subnet: {(2**host_bits)-2}")
    print(f"Subnets Possible: {num_subnets}")

    return network_int, block_size


#  Scan all hosts in subnet
def scan_subnet(network_int, block_size):
    print("\nScanning reachable hosts...")
    reachable = []

    start = network_int + 1
    end   = network_int + block_size - 2

    for ip_int in range(start, end+1):
        ip = int_to_ip(ip_int)
        if ping_host(ip):
            reachable.append(ip)

    return reachable

#  Output
if __name__ == "__main__":

    auto_ip = auto_detect_ip()

    # If auto-detection failed, ask user for IP
    if auto_ip is None:
        print("\nCould not auto-detect IP.")
        print("Please enter IP manually (example: 192.168.1.10/24):")
        auto_ip = input("> ").strip()

    required_hosts = int(input("Required hosts per subnet: "))

    network_int, block_size = calculate_subnet(auto_ip, required_hosts)

    reachable = scan_subnet(network_int, block_size)

    print("\nReachable hosts:")
    print(reachable)


Detecting host IP using Linux tools...
Detected: 172.28.0.12/16
Required hosts per subnet: 10000

Subnet  Network ID    Valid Range                    Broadcast ID  
-------------------------------------------------------------------
1       172.28.0.0    172.28.0.1 - 172.28.63.254     172.28.63.255 
2       172.28.64.0   172.28.64.1 - 172.28.127.254   172.28.127.255
3       172.28.128.0  172.28.128.1 - 172.28.191.254  172.28.191.255
4       172.28.192.0  172.28.192.1 - 172.28.255.254  172.28.255.255

--- Summary ---
Given IP: 172.28.0.12
CIDR: 16
Original Mask: 255.255.0.0
New CIDR: 18
New Mask: 255.255.192.0
Max Hosts/Subnet: 16382
Subnets Possible: 4

Scanning reachable hosts...

Reachable hosts:
[]
